In [ ]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root='./data', train=True, transform=trans, download=True)
test_set = dset.MNIST(root='./data', train=False, transform=trans)
batch_size = 32


train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

In [3]:
class AlexNet(nn.Module):   
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),   
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),
         
        )
    
    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x
    

In [7]:
def train(epoch, data_size = None):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if data_size is not None:
            if batch_idx*batch_size > data_size:
                break
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()                
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
#         loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('\rEpoch: {} {:.0f}%\t     Loss: {:.6f}'.format(
                epoch,
                100. * batch_idx / len(train_loader), loss.item()), end='')

In [8]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()        
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()
    
    
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    acc=100. * float(correct.to(torch.device('cpu')).numpy())
    print('\nTest result: Average loss: {:.4f}, Accuracy: {:.4f}%\n'.format(
        test_loss, acc / len(test_loader.dataset)))
    
    test_accuracy.append(acc / len(test_loader.dataset))

In [9]:
import time

for sz in [1000,3000,5000,10000,50000]:
    model = AlexNet()
    if torch.cuda.is_available():
        model.cuda()

    optimizer = optim.SGD(model.parameters(), lr=0.01)

    start_time = time.time()
    train_losses = []
    test_losses =[]
    test_accuracy = []
    for epoch in range(1, 15):
        train(epoch,sz)
        test()

    print(sz,time.time() - start_time)

    
    

Epoch: 1 0%	     Loss: 2.303334

/tmp/ipykernel_11028/3574512791.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/opt/conda/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test result: Average loss: 2.3030, Accuracy: 9.5800%

Epoch: 2 0%	     Loss: 2.297433
Test result: Average loss: 2.3028, Accuracy: 9.8200%

Epoch: 3 0%	     Loss: 2.302101
Test result: Average loss: 2.3028, Accuracy: 9.5800%

Epoch: 4 0%	     Loss: 2.304344
Test result: Average loss: 2.3026, Accuracy: 9.5800%

Epoch: 5 0%	     Loss: 2.303516
Test result: Average loss: 2.3025, Accuracy: 10.0900%

Epoch: 6 0%	     Loss: 2.302158
Test result: Average loss: 2.3024, Accuracy: 10.0900%

Epoch: 7 0%	     Loss: 2.301932
Test result: Average loss: 2.3022, Accuracy: 10.0900%

Epoch: 8 0%	     Loss: 2.298164
Test result: Average loss: 2.3021, Accuracy: 10.0900%

Epoch: 9 0%	     Loss: 2.304440
Test result: Average loss: 2.3020, Accuracy: 18.8000%

Epoch: 10 0%	     Loss: 2.304947
Test result: Average loss: 2.3020, Accuracy: 11.3500%

Epoch: 11 0%	     Loss: 2.302033
Test result: Average loss: 2.3020, Accuracy: 11.3500%

Epoch: 12 0%	     Loss: 2.299177
Test result: Average loss: 2.3018, Accuracy

In [ ]:
1000 125.25717401504517
3000 189.90918970108032
5000 255.80986189842224
10000 415.30926871299744
50000 1756.6890790462494

In [ ]:
!pip install numpy==1.19.5

In [ ]:
!pip install torch==1.4.0

In [ ]:
!pip install torchvision==0.10.0